In [3]:
from grid import ipfsapi
import base64
import random
import torch
import keras
import json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np 
from grid.grid import Grid

In [4]:
grid = Grid()
grid.id

'Qma5efEQtBUxDonmSDajv9dZ52axH9MNULKTgVpXWK8Ak2'

# Run Worker

In [1]:
# grid.work()

# Run Client

In [5]:
input = np.array([[0,0],[0,1],[1,0],[1,1]])
target = np.array([[0],[1],[1],[0]])

model = Sequential()
model.add(Dense(8, input_dim=2))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.1)
model.compile(loss='binary_crossentropy', optimizer=sgd)

In [15]:
grid.api.dht_findpeer("QmNUNntE3TSnGvhWmbcXaEf3wLdmguUPXrVXtjKQwdtoNZ")

[{'Extra': '',
  'ID': 'QmTKAiy3ic7ff4tGHoes6G4D1TW7PKHdM3Q1ZLoxbJeqpc',
  'Responses': None,
  'Type': 6},
 {'Extra': '',
  'ID': 'QmaqYTK6gJZLh8MMnyDJVzfrG23vjZ2fCKTXtxHCCow3yi',
  'Responses': None,
  'Type': 6},
 {'Extra': '',
  'ID': 'QmWY2wgmwFdobqNrPycmxduMDjCZVdFTzRxvrDcvjV6ZPK',
  'Responses': None,
  'Type': 6},
 {'Extra': '',
  'ID': 'QmWY2wgmwFdobqNrPycmxduMDjCZVdFTzRxvrDcvjV6ZPK',
  'Responses': None,
  'Type': 0},
 {'Extra': '',
  'ID': 'QmTKAiy3ic7ff4tGHoes6G4D1TW7PKHdM3Q1ZLoxbJeqpc',
  'Responses': None,
  'Type': 0},
 {'Extra': '',
  'ID': 'QmaqYTK6gJZLh8MMnyDJVzfrG23vjZ2fCKTXtxHCCow3yi',
  'Responses': None,
  'Type': 0},
 {'Extra': '',
  'ID': 'QmaqYTK6gJZLh8MMnyDJVzfrG23vjZ2fCKTXtxHCCow3yi',
  'Responses': [{'Addrs': ['/ip4/10.12.0.5/tcp/4001',
     '/ip4/127.0.0.1/tcp/4001',
     '/ip4/192.241.204.193/tcp/4001',
     '/ip6/::1/tcp/4001'],
    'ID': 'QmTKAiy3ic7ff4tGHoes6G4D1TW7PKHdM3Q1ZLoxbJeqpc'},
   {'Addrs': ['/ip4/146.115.153.163/tcp/55538',
     '/ip4/192.168.

In [16]:
grid.api.ping("QmNUNntE3TSnGvhWmbcXaEf3wLdmguUPXrVXtjKQwdtoNZ")

[{'Success': True,
  'Text': 'Looking up peer QmNUNntE3TSnGvhWmbcXaEf3wLdmguUPXrVXtjKQwdtoNZ',
  'Time': 0},
 {'Success': False,
  'Text': 'Peer lookup error: routing: not found',
  'Time': 0}]

In [13]:
model,train_spec = grid.fit(model,input,target,epochs=200,log_interval=10)

KeyboardInterrupt: 

In [8]:
train_spec

{'eval_loss': 0.04754955694079399,
 'model_addr': 'QmPnhm6ptXQnJgNUzKhtkaQVWMFux1QVNGcig49FbB3RBt',
 'parent_model': 'QmVFvmoduAQjpU8Z2jZDUh65go6rqfm58fCFhWX6nFfm93',
 'type': 'transact',
 'worker_id': 'Qma5efEQtBUxDonmSDajv9dZ52axH9MNULKTgVpXWK8Ak2'}

In [26]:
len(grid.api.pubsub_peers()['Strings'])

166

In [117]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 8)                 24        
_________________________________________________________________
activation_19 (Activation)   (None, 8)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 9         
_________________________________________________________________
activation_20 (Activation)   (None, 1)                 0         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


# Run Worker

In [21]:
grid.listen_to_channel(grid.fit_worker,'openmined_new_model')

Epoch 1/1
4/4 [==============================] - 0s


In [56]:








model.fit(X, y, batch_size=1, nb_epoch=1000)
print(model.predict_proba(X))

/Users/atrask/anaconda/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1000
4/4 [==============================] - 0s - loss: 0.7856     
Epoch 2/1000
4/4 [==============================] - 0s - loss: 0.7649     
Epoch 3/1000
4/4 [==============================] - 0s - loss: 0.7501     
Epoch 4/1000
4/4 [==============================] - 0s - loss: 0.7418     
Epoch 5/1000
4/4 [==============================] - 0s - loss: 0.7339     
Epoch 6/1000
4/4 [==============================] - 0s - loss: 0.7332     
Epoch 7/1000
4/4 [==============================] - 0s - loss: 0.7306     
Epoch 8/1000
4/4 [==============================] - 0s - loss: 0.7281     
Epoch 9/1000
4/4 [==============================] - 0s - loss: 0.7263     
Epoch 10/1000
4/4 [==============================] - 0s - loss: 0.7239     
Epoch 11/1000
4/4 [==============================] - 0s - loss: 0.7221     
Epoch 12/1000
4/4 [==============================] - 0s - loss: 0.7207     
Epoch 13/1000
4/4 [==============================] - 0s - loss: 0.7191     
Epoch 14/1000
4/4 [==

4/4 [==============================] - 0s - loss: 0.2320     
Epoch 109/1000
4/4 [==============================] - 0s - loss: 0.2281     
Epoch 110/1000
4/4 [==============================] - 0s - loss: 0.2233     
Epoch 111/1000
4/4 [==============================] - 0s - loss: 0.2188     
Epoch 112/1000
4/4 [==============================] - 0s - loss: 0.2145     
Epoch 113/1000
4/4 [==============================] - 0s - loss: 0.2105     
Epoch 114/1000
4/4 [==============================] - 0s - loss: 0.2063     
Epoch 115/1000
4/4 [==============================] - 0s - loss: 0.2023     
Epoch 116/1000
4/4 [==============================] - 0s - loss: 0.1983     
Epoch 117/1000
4/4 [==============================] - 0s - loss: 0.1944     
Epoch 118/1000
4/4 [==============================] - 0s - loss: 0.1905     
Epoch 119/1000
4/4 [==============================] - 0s - loss: 0.1874     
Epoch 120/1000
4/4 [==============================] - 0s - loss: 0.1837     
Epoch 121/1000

4/4 [==============================] - 0s - loss: 0.0262     
Epoch 321/1000
4/4 [==============================] - 0s - loss: 0.0261     
Epoch 322/1000
4/4 [==============================] - 0s - loss: 0.0260     
Epoch 323/1000
4/4 [==============================] - 0s - loss: 0.0258     
Epoch 324/1000
4/4 [==============================] - 0s - loss: 0.0257     
Epoch 325/1000
4/4 [==============================] - ETA: 0s - loss: 0.033 - 0s - loss: 0.0256     
Epoch 326/1000
4/4 [==============================] - 0s - loss: 0.0254     
Epoch 327/1000
4/4 [==============================] - 0s - loss: 0.0253     
Epoch 328/1000
4/4 [==============================] - 0s - loss: 0.0252     
Epoch 329/1000
4/4 [==============================] - 0s - loss: 0.0251     
Epoch 330/1000
4/4 [==============================] - 0s - loss: 0.0249     
Epoch 331/1000
4/4 [==============================] - 0s - loss: 0.0248     
Epoch 332/1000
4/4 [==============================] - 0s - loss: 0.

4/4 [==============================] - 0s - loss: 0.0121     
Epoch 531/1000
4/4 [==============================] - 0s - loss: 0.0121     
Epoch 532/1000
4/4 [==============================] - 0s - loss: 0.0121     
Epoch 533/1000
4/4 [==============================] - 0s - loss: 0.0120     
Epoch 534/1000
4/4 [==============================] - 0s - loss: 0.0120     
Epoch 535/1000
4/4 [==============================] - 0s - loss: 0.0120     
Epoch 536/1000
4/4 [==============================] - 0s - loss: 0.0119     
Epoch 537/1000
4/4 [==============================] - 0s - loss: 0.0119     
Epoch 538/1000
4/4 [==============================] - 0s - loss: 0.0119     
Epoch 539/1000
4/4 [==============================] - 0s - loss: 0.0118     
Epoch 540/1000
4/4 [==============================] - 0s - loss: 0.0118     
Epoch 541/1000
4/4 [==============================] - 0s - loss: 0.0118     
Epoch 542/1000
4/4 [==============================] - 0s - loss: 0.0117     
Epoch 543/1000

4/4 [==============================] - 0s - loss: 0.0076     
Epoch 742/1000
4/4 [==============================] - 0s - loss: 0.0076     
Epoch 743/1000
4/4 [==============================] - 0s - loss: 0.0075     
Epoch 744/1000
4/4 [==============================] - 0s - loss: 0.0075     
Epoch 745/1000
4/4 [==============================] - 0s - loss: 0.0075     
Epoch 746/1000
4/4 [==============================] - 0s - loss: 0.0075     
Epoch 747/1000
4/4 [==============================] - 0s - loss: 0.0075     
Epoch 748/1000
4/4 [==============================] - 0s - loss: 0.0075     
Epoch 749/1000
4/4 [==============================] - 0s - loss: 0.0075     
Epoch 750/1000
4/4 [==============================] - 0s - loss: 0.0075     
Epoch 751/1000
4/4 [==============================] - 0s - loss: 0.0074     
Epoch 752/1000
4/4 [==============================] - 0s - loss: 0.0074     
Epoch 753/1000
4/4 [==============================] - 0s - loss: 0.0074     
Epoch 754/1000

4/4 [==============================] - 0s - loss: 0.0054     
Epoch 953/1000
4/4 [==============================] - 0s - loss: 0.0054     
Epoch 954/1000
4/4 [==============================] - 0s - loss: 0.0054     
Epoch 955/1000
4/4 [==============================] - 0s - loss: 0.0054     
Epoch 956/1000
4/4 [==============================] - 0s - loss: 0.0054     
Epoch 957/1000
4/4 [==============================] - 0s - loss: 0.0054     
Epoch 958/1000
4/4 [==============================] - 0s - loss: 0.0054     
Epoch 959/1000
4/4 [==============================] - 0s - loss: 0.0054     
Epoch 960/1000
4/4 [==============================] - 0s - loss: 0.0053     
Epoch 961/1000
4/4 [==============================] - 0s - loss: 0.0053     
Epoch 962/1000
4/4 [==============================] - 0s - loss: 0.0053     
Epoch 963/1000
4/4 [==============================] - 0s - loss: 0.0053     
Epoch 964/1000
4/4 [==============================] - 0s - loss: 0.0053     
Epoch 965/1000

In [7]:
def process_message(decoded):
    # send some sort of message
    try:
        # if message is a number... reply with the number + 1
        api.pubsub_pub(topic='openmined_model_1',payload=int(decoded['data']) + 1)
    except:
        print("could not accumulate... message not an int")
    print(decoded['data'])

# connect


my_id = ""
secret = random.randint(0,1000000)

# send secret identity


results = list()

for encoded in g:
    
    if('from' in encoded):
        # decode message
        decoded = {}
        decoded['from'] = base64.standard_b64decode(encoded['from'])
        decoded['data'] = base64.standard_b64decode(encoded['data']).decode('ascii')
        decoded['seqno'] = base64.standard_b64decode(encoded['seqno'])
        decoded['topicIDs'] = encoded['topicIDs']
        decoded['encoded'] = encoded
        results.append(decoded)

        # If I don't know who I am - wait for id
        if(my_id == ""):
            if(str(decoded['data'].split(":")[-1]) == str(secret)):
                my_id = str(decoded['from'])
                print("Id:" + my_id)

        # if I know who i am - process message
        else:

            if(str(decoded['from']) != my_id and decoded['data'][0:3] != 'id:'):
                process_message(decoded)


Id:b'\x12 \xaety\x95\xb0\xeb\xd2\xb3f\xc9\xe1\xa9\x87\xf8(\xb0\x98cm\xa4\xd3=\xf3\x9a\xeax~\xce\x05mX['


KeyboardInterrupt: 